In [2]:
import yaml
from myclient import MyClient
import json
from rich import print
import requests
myclient=MyClient(logging=False)

In [3]:
link="https://cad.onshape.com/documents/1ffb81a71e5b402e966b9341/w/6e295017d1b34be684565c40/e/bb398e4615fe4025b34ea8f0"

v_list = link.split("/")
did, wid, eid = v_list[-5], v_list[-3], v_list[-1]

In [14]:
myclient.eval_sketch_topology_by_adjacency(did, wid, eid, feat_id="FV77uVJcfQfEYIM")

{'edges': [{'id': 'JKB',
   'param': {'type': 'Circle',
    'coordSystem': {'origin': [-0.0322322, 0.0, -0.0086014],
     'xAxis': [0.0, 0.0, -1.0],
     'zAxis': [0.0, -1.0, 0.0]},
    'radius': 0.00561975},
   'vertices': ['IK', 'JKE']},
  {'id': 'JKF',
   'param': {'type': 'Line',
    'direction': [0.0, 0.0, -1.0],
    'origin': [-0.0322322, 0.0, -0.00298165]},
   'vertices': ['IK', 'JKE']}],
 'faces': [{'edges': ['JKB', 'JKF'],
   'id': 'JKC',
   'param': {'type': 'Plane',
    'normal': [0.0, -1.0, 0.0],
    'origin': [-0.02940546, 0.0, -0.0086014],
    'x': [1.0, 0.0, 0.0]}}],
 'vertices': [{'id': 'IK',
   'param': {'Vector': (-0.0322322, 0.0, -0.01422115), 'unit': ('METER', 1)}},
  {'id': 'JKE',
   'param': {'Vector': (-0.0322322, 0.0, -0.00298165), 'unit': ('METER', 1)}}]}

In [6]:
with open("test.json","r") as f:
    jsonData=json.load(f)

with open("creds.json","r") as f:
    credData=json.load(f)

In [17]:
i=1
print(jsonData['features'][i]['message']['featureId'],jsonData['features'][i]['message']['featureType'])
feature_data=jsonData['features'][i]['message']['parameters']
get_param_dict(feature_data)

FSKvoiZjdOfBlyo revolve

{'bodyType': 'SOLID',
 'operationType': 'NEW',
 'entities': ['JGC'],
 'surfaceEntities': [],
 'axis': ['JFB'],
 'revolveType': 'FULL',
 'oppositeDirection': False,
 'angle': '30.0*deg',
 'angleBack': '30.0*deg',
 'defaultScope': False,
 'booleanScope': [],
 'asVersion': 'V137_VERSION_HISTORY_ENTRY'}

In [8]:
def get_param_dict(feature):
    param_dict={}
    for _,param_item in enumerate(feature_data):
        param_msg = param_item['message']
        param_id = param_msg['parameterId']

        if "queries" in param_msg:
            param_value=[]
            for j in range(len(param_msg['queries'])):
                # Values look like  "JGG"
                param_value.extend(param_msg['queries'][j]['message']['geometryIds'])
        # Values look like '30.0*deg'
        elif 'expression' in param_msg:
            param_value = param_msg['expression']
        elif 'value' in param_msg:
            param_value = param_msg['value']
        else:
            raise NotImplementedError('param_msg:\n{}'.format(param_msg))

        param_dict.update({param_id: param_value})
    return param_dict

In [64]:
param_dict

{'entities': ['JQK', 'JQG'],
 'radius': '0.05*in',
 'tangentPropagation': True,
 'rho': '0.5',
 'asVersion': 'V608_MERGE_FROM_TOOLS',
 'allowEdgeOverflow': False}

In [18]:
myclient=MyClient()
myclient.expr2meter(did,wid,eid,"0.1*in")

2023-07-16 19:13:28.034 | INFO     | apikey.onshape:__init__:80 - onshape instance created: url = https://cad.onshape.com, access key = b'kr6Z1LgdgLrlX8nchk9qE5s9'
2023-07-16 19:13:28.035 | INFO     | apikey.onshape:_make_nonce:94 - nonce created: T0v8j6vcS7zDvyDw0POtcfdg0
2023-07-16 19:13:28.035 | INFO     | apikey.onshape:_make_auth:124 - {'query': '', 'hmac_str': b'post\nt0v8j6vcs7zdvydw0potcfdg0\nsun, 16 jul 2023 17:13:28 gmt\napplication/json\n/api/partstudios/d/1ffb81a71e5b402e966b9341/w/6e295017d1b34be684565c40/e/bb398e4615fe4025b34ea8f0/featurescript\n\n', 'signature': b'gNMzRcXOoliuUJrjVxTX76ftMDhsb0uVNfAthhLJDM8=', 'auth': 'On kr6Z1LgdgLrlX8nchk9qE5s9:HmacSHA256:gNMzRcXOoliuUJrjVxTX76ftMDhsb0uVNfAthhLJDM8='}
2023-07-16 19:13:28.036 | INFO     | apikey.onshape:request:186 - {'script': 'function(context is Context, queries) {    return lookupTableEvaluate("0.1*in") * meter;}', 'queries': []}
2023-07-16 19:13:28.036 | INFO     | apikey.onshape:request:187 - {'Content-Type': 'app

0.0025400000000000006

In [19]:
def expr2radians(did, wid, eid, expr):
    """convert value expresson to meter unit"""
    body = {
        "script":
            "function(context is Context, queries) { "
            "   return lookupTableEvaluate(\"%s\") * deg;" % (expr) +
            "}",
        "queries": []
    }

    res = self._api.request('post', '/api/partstudios/d/' + did + '/w/' + wid + '/e/' + eid +
                            '/featurescript', body=body).json()
    
    return res['result']['message']['value']

In [9]:
expr2radians(did,wid,eid,)

{'a': 13, 'b': 'ae'}

In [22]:
import requests

# Specify the API endpoint for retrieving document information
url = f"https://cad.onshape.com/api/documents/{did}/"

# Make a GET request to retrieve the document details
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Get the JSON response
    data = response.json()

    # Check if the document has lookup table variables
    if "lookupTables" in data:
        # Get the lookup table variables
        lookup_tables = data["lookupTables"]

        # Print the lookup table variables
        for variable in lookup_tables:
            variable_name = variable["name"]
            print(f"Lookup Table Variable: {variable_name}")

    else:
        print("No lookup table variables found in the document.")

else:
    print("Failed to retrieve document information.")


No lookup table variables found in the document.

In [23]:
data

{'message': 'Resource does not exist, or you do not have permission to access it.',
 'moreInfoUrl': None,
 'status': 403,
 'code': 1002}